In [13]:
from scipy.sparse import csr_matrix
import scipy as sp
import numpy as np
trainUser = np.array([1,1,2,2,2,3]) - 1
trainItem = np.array([4,5,1,3,4,2]) - 1
n_user = 3
m_item = 5
UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),
                                      shape=(n_user, m_item))
print(UserItemNet)

  (0, 3)	1.0
  (0, 4)	1.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 3)	1.0
  (2, 1)	1.0


In [19]:
highpo_samples = np.array([2,3])
rows = np.arange(n_user)[:,None]
rows = np.repeat(rows,len(highpo_samples),axis=1).flatten()

In [22]:
cols = np.tile(highpo_samples, n_user)


In [24]:
rows

array([0, 0, 1, 1, 2, 2])

In [23]:
cols

array([2, 3, 2, 3, 2, 3])

In [26]:
data = np.ones_like(cols)
data

array([1, 1, 1, 1, 1, 1])

In [28]:
user_popularity = csr_matrix((data, (rows, cols)), shape=(n_user, m_item))

In [30]:
print(user_popularity)

  (0, 2)	1
  (0, 3)	1
  (1, 2)	1
  (1, 3)	1
  (2, 2)	1
  (2, 3)	1


In [32]:
user_popularity = user_popularity.multiply(UserItemNet)
print(user_popularity)

  (0, 3)	1.0
  (1, 2)	1.0
  (1, 3)	1.0


In [7]:
highpo_samples = np.array([2,3])
user_popularity[highpo_samples] = 1

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [3]:
UserItemNet[0].nonzero()[1]

array([3, 4], dtype=int32)

In [4]:
posItems = []
posItems.append(UserItemNet[0].nonzero()[1])

In [6]:
posItems.append(UserItemNet[1].nonzero()[1])
posItems

[array([3, 4], dtype=int32), array([0, 2, 3], dtype=int32)]

In [12]:
UserItemNet[0,3].__getattribute__

<method-wrapper '__getattribute__' of numpy.float64 object at 0x7fb0e284bb90>

In [12]:
import scipy.sparse as sp
adj_mat = sp.dok_matrix((n_user + m_item, n_user + m_item), dtype=np.float32)
adj_mat = adj_mat.tolil()

In [19]:
a

<8x8 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Dictionary Of Keys format>

In [13]:
R = UserItemNet.tolil()

In [14]:
adj_mat[:n_user, n_user:] = R
adj_mat[n_user:, :n_user] = R.T


In [15]:
adj_mat = adj_mat.todok()

In [33]:
print(adj_mat)

  (6, 0)	1.0
  (7, 0)	1.0
  (3, 1)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (4, 2)	1.0
  (1, 3)	1.0
  (2, 4)	1.0
  (1, 5)	1.0
  (0, 6)	1.0
  (1, 6)	1.0
  (0, 7)	1.0


In [16]:
rowsum = np.array(adj_mat.sum(axis=1))
rowsum

array([[2.],
       [3.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [1.]], dtype=float32)

In [17]:
d_inv = np.power(rowsum, -0.5).flatten()
d_inv

array([0.70710677, 0.57735026, 1.        , 1.        , 1.        ,
       1.        , 0.70710677, 1.        ], dtype=float32)

In [18]:
d_mat = sp.diags(d_inv)
print(d_mat)

  (0, 0)	0.70710677
  (1, 1)	0.57735026
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	0.70710677
  (7, 7)	1.0


In [19]:
norm_adj = d_mat.dot(adj_mat)
print(norm_adj)

  (0, 7)	0.70710677
  (0, 6)	0.70710677
  (1, 6)	0.57735026
  (1, 5)	0.57735026
  (1, 3)	0.57735026
  (2, 4)	1.0
  (3, 1)	1.0
  (4, 2)	1.0
  (5, 1)	1.0
  (6, 1)	0.70710677
  (6, 0)	0.70710677
  (7, 0)	1.0


In [20]:
norm_adj = norm_adj.dot(d_mat)
print(norm_adj)

  (0, 6)	0.49999997
  (0, 7)	0.70710677
  (1, 3)	0.57735026
  (1, 5)	0.57735026
  (1, 6)	0.40824828
  (2, 4)	1.0
  (3, 1)	0.57735026
  (4, 2)	1.0
  (5, 1)	0.57735026
  (6, 0)	0.49999997
  (6, 1)	0.40824828
  (7, 0)	0.70710677


In [21]:
norm_adj = norm_adj.tocsr()
print(norm_adj)

  (0, 6)	0.49999997
  (0, 7)	0.70710677
  (1, 3)	0.57735026
  (1, 5)	0.57735026
  (1, 6)	0.40824828
  (2, 4)	1.0
  (3, 1)	0.57735026
  (4, 2)	1.0
  (5, 1)	0.57735026
  (6, 0)	0.49999997
  (6, 1)	0.40824828
  (7, 0)	0.70710677


In [22]:
X = norm_adj
coo = X.tocoo().astype(np.float32)

In [27]:
import torch
coo = X.tocoo().astype(np.float32)
row = torch.Tensor(coo.row).long()
col = torch.Tensor(coo.col).long()
index = torch.stack([row, col])
data = torch.FloatTensor(coo.data)

In [32]:
a = torch.sparse.FloatTensor(index, data, torch.Size(coo.shape)).coalesce()

In [38]:
keep_prob = 0.6

In [39]:
random_index = torch.rand(len(a.values())) + keep_prob
random_index

tensor([1.5051, 1.4183, 1.5672, 1.1399, 0.9426, 1.4170, 1.3702, 1.5139, 1.3634,
        0.8299, 1.0084, 1.3435])

In [41]:
random_index = random_index.int().bool()

In [43]:
index = a.indices().t()
index = index[random_index]
index

tensor([[0, 6],
        [0, 7],
        [1, 3],
        [1, 5],
        [2, 4],
        [3, 1],
        [4, 2],
        [5, 1],
        [6, 1],
        [7, 0]])

In [44]:
a.indices().t()

tensor([[0, 6],
        [0, 7],
        [1, 3],
        [1, 5],
        [1, 6],
        [2, 4],
        [3, 1],
        [4, 2],
        [5, 1],
        [6, 0],
        [6, 1],
        [7, 0]])

In [47]:
import torch
row = torch.Tensor(coo.row).long()
col = torch.Tensor(coo.col).long()
index = torch.stack([row, col])

In [ ]:
index[0]

tensor([[0, 0, 1, 1, 1, 2, 3, 4, 5, 6, 6, 7],
        [6, 7, 3, 5, 6, 4, 1, 2, 1, 0, 1, 0]])

In [4]:
import numpy as np
batch_size = 5
users = np.random.randint(0,10,5)
pos = np.random.randint(0,5,5)
neg = np.random.randint(0,20,5)

In [6]:
import torch
n_dim = 5
n_user = 5
m_item = 3
embedding_user = torch.nn.Embedding(n_user, n_dim).weight
embedding_item = torch.nn.Embedding(m_item, n_dim).weight

In [7]:
torch.mul(embedding_user[0],embedding_item[0])

tensor([ 0.4374,  0.5484,  0.4191, -1.2630, -0.0174], grad_fn=<MulBackward0>)

In [49]:
all_emb = torch.cat([embedding_user, embedding_item])
all_emb

tensor([[-0.3642, -0.4743, -1.2291,  0.3904, -0.1371],
        [ 0.5780,  2.0548, -1.7331, -1.1129,  0.8137],
        [ 0.3092,  0.5179,  1.0910, -0.4013, -1.4377],
        [-0.1943,  1.1920,  0.2614,  0.5418, -1.9513],
        [ 0.3853, -0.0947,  0.1074, -0.1851,  1.2398],
        [-0.4665, -1.1952,  0.6807, -1.0512,  0.4131],
        [ 0.4097,  0.3224,  1.3734, -0.7116, -0.5134],
        [ 1.4123, -0.9352,  0.2088, -1.4172,  1.1381]], grad_fn=<CatBackward0>)

In [50]:
all_emb = torch.sparse.mm(a, all_emb)
all_emb

tensor([[ 1.2035, -0.5001,  0.8344, -1.3579,  0.5481],
        [-0.2142,  0.1298,  1.1046, -0.5846, -1.0976],
        [ 0.3853, -0.0947,  0.1074, -0.1851,  1.2398],
        [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
        [ 0.3092,  0.5179,  1.0910, -0.4013, -1.4377],
        [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
        [ 0.0539,  0.6017, -1.3221, -0.2591,  0.2636],
        [-0.2575, -0.3354, -0.8691,  0.2761, -0.0970]],
       grad_fn=<SparseAddmmBackward0>)

In [54]:
b = []

In [55]:
b.append(all_emb)
b.append(all_emb)

In [56]:
b

[tensor([[ 1.2035, -0.5001,  0.8344, -1.3579,  0.5481],
         [-0.2142,  0.1298,  1.1046, -0.5846, -1.0976],
         [ 0.3853, -0.0947,  0.1074, -0.1851,  1.2398],
         [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
         [ 0.3092,  0.5179,  1.0910, -0.4013, -1.4377],
         [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
         [ 0.0539,  0.6017, -1.3221, -0.2591,  0.2636],
         [-0.2575, -0.3354, -0.8691,  0.2761, -0.0970]],
        grad_fn=<SparseAddmmBackward0>),
 tensor([[ 1.2035, -0.5001,  0.8344, -1.3579,  0.5481],
         [-0.2142,  0.1298,  1.1046, -0.5846, -1.0976],
         [ 0.3853, -0.0947,  0.1074, -0.1851,  1.2398],
         [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
         [ 0.3092,  0.5179,  1.0910, -0.4013, -1.4377],
         [ 0.3337,  1.1863, -1.0006, -0.6425,  0.4698],
         [ 0.0539,  0.6017, -1.3221, -0.2591,  0.2636],
         [-0.2575, -0.3354, -0.8691,  0.2761, -0.0970]],
        grad_fn=<SparseAddmmBackward0>)]

In [60]:
b = torch.stack(b,dim=1)

In [2]:
light_out = torch.mean(b, dim=1)

NameError: name 'torch' is not defined

In [1]:
light_out[0]

NameError: name 'light_out' is not defined

In [ ]:
light_out[0]

In [63]:
tensor1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor2 = torch.tensor([[7, 8, 9], [10, 11, 12]])

# 使用 torch.stack() 函数将两个张量堆叠在一起
stacked_tensor = torch.stack([tensor1, tensor2],dim=1)
stacked_tensor

tensor([[[ 1,  2,  3],
         [ 7,  8,  9]],

        [[ 4,  5,  6],
         [10, 11, 12]]])

In [ ]:
users, items = torch.split(light_out, [self.num_users, self.num_items])

In [1]:
import torch
batch_size = 5
voc = 10
rank = torch.randn(batch_size,voc)

In [2]:
rank


tensor([[ 1.2008, -1.6609,  1.9343,  1.0009, -0.5086,  0.7318,  0.6679, -0.0940,
          0.2518, -1.5456],
        [ 0.9686,  0.2994,  1.3507, -0.1406,  0.8614, -0.0312,  0.1206,  0.5280,
         -0.6347,  0.6546],
        [-0.3691,  0.7874,  0.2441, -0.1598,  1.4536, -1.1945, -0.8731,  0.7878,
         -0.0754, -0.4170],
        [-0.7047, -2.2025,  1.7144,  0.4498,  1.1371,  0.5230, -0.6301,  1.5473,
         -1.1550, -0.7806],
        [ 0.3932, -1.6229, -1.2678, -0.1566,  1.7816,  0.8374,  0.2671,  0.0594,
         -0.8235, -0.1192]])

In [3]:
a,b = torch.topk(rank, k=5)

In [4]:
a

tensor([[ 1.9343,  1.2008,  1.0009,  0.7318,  0.6679],
        [ 1.3507,  0.9686,  0.8614,  0.6546,  0.5280],
        [ 1.4536,  0.7878,  0.7874,  0.2441, -0.0754],
        [ 1.7144,  1.5473,  1.1371,  0.5230,  0.4498],
        [ 1.7816,  0.8374,  0.3932,  0.2671,  0.0594]])

In [5]:
b

tensor([[2, 0, 3, 5, 6],
        [2, 0, 4, 9, 7],
        [4, 7, 1, 2, 8],
        [2, 7, 4, 5, 3],
        [4, 5, 0, 6, 7]])

In [8]:
import numpy as np
a = np.random.randn(3, 5)


In [14]:
a[0,:2]=1

In [15]:
a

array([[ 1.        ,  1.        ,  2.58089788,  0.36540889,  1.        ],
       [ 1.81336212,  0.14038956,  0.64086827,  0.96425241,  1.        ],
       [-1.22044359, -1.11297507,  0.11281308, -0.47448022,  1.        ]])

In [9]:
import numpy as np
k = 5
b =  1./np.log2(np.arange(2, k + 2))

In [10]:
b

array([1.        , 0.63092975, 0.5       , 0.43067656, 0.38685281])

In [11]:
a * b

array([[-1.26063632, -0.57880695,  1.29044894,  0.15737304,  0.02228321],
       [ 1.81336212,  0.08857595,  0.32043414,  0.41528091,  0.06883264],
       [-1.22044359, -0.70220909,  0.05640654, -0.20434751, -0.13735176]])

In [4]:
import numpy as np
item_popularity = np.array([4,3,5,1,2,0,6,7])
threshold = 3
positive_samples = np.where(item_popularity > threshold)
negative_samples = item_popularity[item_popularity <= threshold]

In [5]:
positive_samples

(array([0, 2, 6, 7]),)

In [1]:
import torch
anchor_emb = torch.randn(5, 10)
pos_emb = torch.randn(5, 10)
a = torch.matmul(anchor_emb, pos_emb.T)